In [5]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from datetime import datetime

In [7]:
# 데이터 전처리

'''
# 데이터셋 로드 및 전처리
file_path = '/content/korean_emotional.xlsx'
data = pd.read_excel(file_path)
data = data[['Sentence', 'Emotion']].dropna()

# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['Emotion'] = label_encoder.fit_transform(data['Emotion'])

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['Sentence'], data['Emotion'], test_size=0.2, random_state=42
)
'''
'''
# 1. 데이터 로드 및 전처리
file_path = '/Users/kyunghwanoh/Project/test/korean_emotional.xlsx'
data = pd.read_excel(file_path)
data = data[['Sentence', 'Emotion']].dropna()

# 라벨 인코딩
label_encoder = LabelEncoder()
data['Emotion'] = label_encoder.fit_transform(data['Emotion'])

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['Sentence'], data['Emotion'], test_size=0.2, random_state=42
)

'''

train_file_path = '/content/total_train.csv'
validation_file_path = '/content/total_validation.csv'

# Train 데이터 로드
train_data = pd.read_csv(train_file_path)
train_data = train_data[['Sentence', 'Emotion']].dropna()

# Validation 데이터 로드
validation_data = pd.read_csv(validation_file_path)
validation_data = validation_data[['Sentence', 'Emotion']].dropna()

# 라벨 인코딩
label_encoder = LabelEncoder()
train_data['Emotion'] = label_encoder.fit_transform(train_data['Emotion'])
validation_data['Emotion'] = label_encoder.transform(validation_data['Emotion'])

# 각각의 텍스트와 라벨 분리
train_texts, train_labels = train_data['Sentence'], train_data['Emotion']
val_texts, val_labels = validation_data['Sentence'], validation_data['Emotion']

# 데이터 샘플 확인
print("Train Data Sample:")
print(train_texts.head())
print(train_labels.head())

print("\nValidation Data Sample:")
print(val_texts.head())
print(val_labels.head())


Train Data Sample:
0    언니 동생으로 부르는게 맞는 일인가요..??
1                그냥 내 느낌일뿐겠지?
2              아직너무초기라서 그런거죠?
3               유치원버스 사고 낫다던데
4                 근데 원래이런거맞나요
Name: Sentence, dtype: object
0    0
1    0
2    0
3    0
4    0
Name: Emotion, dtype: int64

Validation Data Sample:
0    이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...
1    회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.큰 프로젝트를 혼...
2    상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.직장 상사가 무섭게 생기셔서 친...
3    이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.첫 직장이...
4    직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.직장 사람들 관계에 대해서 고민이...
Name: Sentence, dtype: object
0    0
1    0
2    0
3    0
4    0
Name: Emotion, dtype: int64


In [8]:
'''# Tokenizer 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 텍스트 토크나이징
def tokenize_data(texts, labels, tokenizer, max_length=128):
    # 리스트 형태로 변환
    texts = texts.tolist() if isinstance(texts, pd.Series) else texts
    labels = labels.tolist() if isinstance(labels, pd.Series) else labels

    # 토큰화 처리
    encoding = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt",
        clean_up_tokenization_spaces=True  # 경고 제거
    )
    dataset = torch.utils.data.TensorDataset(
        encoding['input_ids'],
        encoding['attention_mask'],
        torch.tensor(labels)
    )
    return dataset

# 데이터셋 준비
train_dataset = tokenize_data(train_texts, train_labels, tokenizer)
val_dataset = tokenize_data(val_texts, val_labels, tokenizer)
'''

# 2. Hugging Face Tokenizer 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 3. 데이터셋 변환 함수
def convert_to_hf_dataset(texts, labels, tokenizer, max_length=128):
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=max_length
    )
    hf_dataset = Dataset.from_dict({
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": labels
    })
    return hf_dataset

    # Hugging Face Dataset으로 변환
train_dataset = convert_to_hf_dataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
val_dataset = convert_to_hf_dataset(val_texts.tolist(), val_labels.tolist(), tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [9]:
#  모델 선정
"""
from transformers import BertForSequenceClassification

# 모델 로드
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',  # 멀티언어 지원 모델
    num_labels=len(label_encoder.classes_)  # 감정 레이블 수에 따라 출력 설정
)
"""
# 4. 모델 로드
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=len(label_encoder.classes_)
)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# 모댈 학습 설정

# from transformers import TrainingArguments, Trainer
'''
# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',           # 모델 저장 위치
    evaluation_strategy="epoch",     # 에폭마다 평가
    learning_rate=2e-5,              # 학습률
    per_device_train_batch_size=16,  # 학습 배치 크기
    per_device_eval_batch_size=16,   # 평가 배치 크기
    num_train_epochs=3,              # 에폭 수
    weight_decay=0.01,               # 가중치 감쇠
    logging_dir='./logs',            # 로깅 디렉토리
    logging_steps=10                 # 로깅 빈도
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)
'''

# 오늘 날짜를 YYYYMMDD 형식으로 설정
today_date = datetime.now().strftime("%Y%m%d")

batch_size = 32
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir=f'/Users/kyunghwanoh/Project/notebook/weights/class_{today_date}',  # output_dir 설정
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,  # 최대 저장 수를 2개로 제한
    logging_steps=10,
    report_to="none"  # wandb 비활성화
)

# 6. Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


In [11]:
# 모델 학습
trainer.train()


Epoch,Training Loss,Validation Loss
1,1.099400,0.683169


Epoch,Training Loss,Validation Loss
1,1.099400,0.683169
2,1.014100,0.656344
3,0.917100,0.642440


TrainOutput(global_step=7296, training_loss=1.0403271114224928, metrics={'train_runtime': 5445.9748, 'train_samples_per_second': 42.867, 'train_steps_per_second': 1.34, 'total_flos': 1.53567714598272e+16, 'train_loss': 1.0403271114224928, 'epoch': 3.0})

In [17]:
#예측 및 활용
'''def predict(text, model, tokenizer):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to("mps")
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return label_encoder.inverse_transform([probs.argmax().item()])
'''

def predict(text, model, tokenizer):
    # 디바이스 설정 (GPU가 있으면 GPU 사용, 없으면 CPU)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # 입력 데이터 토큰화 및 디바이스 할당
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(device)

    # 모델을 디바이스로 이동
    model = model.to(device)

    # 예측 수행
    with torch.no_grad():
        outputs = model(**inputs)

    # 소프트맥스 확률 계산
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # 예측된 레이블 반환
    return label_encoder.inverse_transform([probs.argmax().item()])

# 예시
new_text = input()
predicted_emotion = predict(new_text, model, tokenizer)
print(f"예측 감정: {predicted_emotion}")


너가 지금 뭘 잘했다고 말대꾸야
예측 감정: ['혐오']


---
# 학습된 마지막 모델 불러오기

True
True


가장 최근 체크포인트: /Users/kyunghwanoh/Project/notebook/weights/class_241117/checkpoint-965
사용 장치: mps


/opt/anaconda3/envs/py311/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


입력 텍스트:   진짜 짜증


예측 감정: ['분노']
